## N-Loss Strategy Bot

Get the key element position

Get the screenshot the multiplier position and convert to grayscale

Convert to grayscale image to text using tesseract

Apply n-loss strategy

In [1]:
import pyautogui
from PIL import Image,ImageOps
import pytesseract
import time
import pandas as pd
import copy
import re

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'D:\Tesseract-OCR\tesseract'

In [3]:
#locate key element on roobet page
#note have to have page open on screen
key_element_location = pyautogui.locateOnScreen('key_element.PNG', grayscale=True)

left = key_element_location.left + 35
top = key_element_location.top
width = key_element_location.width + 22
height = key_element_location.height

In [6]:
game_multipliers = []
game_results = [] #losses are True, wins are False
prev_multiplier = 0
death_stop = False
live_game_num = 0
n_loss_games = 4 #4 losses before playing
loss_level = 2

bet_size = 0.01
auto_cashout_level = 1.5

bet_placed = False
betting_results = pd.DataFrame([])

death_count = 0
death_stop_count = 2

while not(death_stop):

    #poll and wait till game ends before recording
    try:
        preparing_round_x, preparing_round_y = pyautogui.locateCenterOnScreen('preparing_round_action.PNG')
    except TypeError:
        #typeError occurs because the round is in action wait till its done 
        continue

    multiplier_box = pyautogui.screenshot(region=(left, top, width, height))
    multiplier_grayscale = ImageOps.grayscale(multiplier_box)

    try:
        roobet_game_text = pytesseract.image_to_string(multiplier_grayscale,config='--psm 8')
        raw_multiplier = re.sub(r'[^\d]+', '', roobet_game_text) 
        multiplier = float(raw_multiplier[:-2] + "." + raw_multiplier[-2:])
    except ValueError:
        #conversion fail try again
        continue
    
    if prev_multiplier != multiplier:

        game_result = loss_level > multiplier
        game_multipliers.append(multiplier)
        game_results.append(game_result)
        print(game_multipliers)
        print(game_results)

        prev_multiplier = multiplier
        live_game_num += 1
        
        #check if initial window is built up
        if (live_game_num >= n_loss_games) and not(bet_placed):
            current_game_window = game_results[-n_loss_games:]
            if sum(current_game_window) == n_loss_games:
                #n games were lost in a row
                #play current game

                #add bet size
                bet_amount_box_x, bet_amount_box_y = pyautogui.locateCenterOnScreen('bet_amount_box.PNG')
                pyautogui.click(bet_amount_box_x+15, bet_amount_box_y+15)
                #delete what is currently there
                pyautogui.hotkey('ctrl', 'a', 'backspace')
                pyautogui.write(str(bet_size))  

                #add cashout level
                auto_cashout_level_x, auto_cashout_level_y = pyautogui.locateCenterOnScreen('auto_cashout_level_box.PNG')
                pyautogui.click(auto_cashout_level_x+15, auto_cashout_level_y+15)
                #delete what is currently there
                pyautogui.hotkey('ctrl', 'a', 'backspace')
                pyautogui.write(str(auto_cashout_level))  

                #click place bet button
                place_bet_x, place_bet_y = pyautogui.locateCenterOnScreen('place_bet_button.PNG')
                pyautogui.click(place_bet_x,place_bet_y)

                #bet placed
                bet_placed = True
                print('Bet placed')
                continue
    
        if bet_placed:
            game_details = pd.Series({'game_no':live_game_num,'bet_size':bet_size})
            if multiplier < auto_cashout_level:
                game_details["outcome"] = 0
                death_count += 1
                print('Lost')
            else:
                game_details["outcome"] = bet_size * multiplier
                death_count = 0
                print('Win')

            bet_placed = False
            #reset window
            game_multipliers = []
            game_results = []

            betting_results = betting_results.append(game_details,ignore_index=True)

    if death_count == death_stop_count:
        death_stop = True

        
        

[3.5]
[False]
[3.5, 1.81]
[False, True]
[3.5, 1.81, 2.5]
[False, True, False]
[3.5, 1.81, 2.5, 1.59]
[False, True, False, True]
[3.5, 1.81, 2.5, 1.59, 1.0]
[False, True, False, True, True]
[3.5, 1.81, 2.5, 1.59, 1.0, 1.34]
[False, True, False, True, True, True]
[3.5, 1.81, 2.5, 1.59, 1.0, 1.34, 1.0]
[False, True, False, True, True, True, True]
Bet placed
[3.5, 1.81, 2.5, 1.59, 1.0, 1.34, 1.0, 5.58]
[False, True, False, True, True, True, True, False]
Win
[1.75]
[True]
[1.75, 1.06]
[True, True]
[1.75, 1.06, 3.17]
[True, True, False]
[1.75, 1.06, 3.17, 9.24]
[True, True, False, False]
[1.75, 1.06, 3.17, 9.24, 60.19]
[True, True, False, False, False]
[1.75, 1.06, 3.17, 9.24, 60.19, 1.12]
[True, True, False, False, False, True]
[1.75, 1.06, 3.17, 9.24, 60.19, 1.12, 1.59]
[True, True, False, False, False, True, True]
[1.75, 1.06, 3.17, 9.24, 60.19, 1.12, 1.59, 0.17]
[True, True, False, False, False, True, True, True]
[1.75, 1.06, 3.17, 9.24, 60.19, 1.12, 1.59, 0.17, 1.81]
[True, True, False,